In [1]:
# Tool setup
Laoding extensions for data explroation

In [1]:
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab, 1.0.0-beta.21215.5"

In [1]:
# Import logs
Use `#!value` to load raw json

In [1]:
#!value --name user_feedback --from-file ./report_export.json

#!value --name server_logs --from-file ./serverlog.json

In [1]:
#!share --from value user_feedback

#!share --from value server_logs

In [1]:
using System.Text.Json;

var user_feedback_json = JsonDocument.Parse(user_feedback);
var server_logs_json = JsonDocument.Parse(server_logs);

In [1]:
# Explore data
Using nteract to explore json files

In [1]:
user_feedback_json.ExploreWithNteract().Display();

In [1]:
server_logs_json.ExploreWithNteract().Display();

In [1]:
server_logs_json.ExploreWithSandDance().Display();

In [1]:
#r "nuget:Microsoft.DotNet.Interactive.SqlServer,*-*"

In [1]:
#!connect mssql --kernel-name adventureworks --create-dbcontext "Persist Security Info=False; Integrated Security=true; Initial Catalog=AdventureWorks2019; Server=localhost"

In [1]:
#!sql-adventureworks
SELECT 
product.ProductID,
product.Name,
category.ProductCategoryID,
product.ProductSubcategoryID,
category.Name
FROM AdventureWorks2019.Production.Product as product
INNER JOIN AdventureWorks2019.Production.ProductCategory as category on product.ProductSubcategoryID = category.ProductCategoryID

In [1]:
var reportedProducts = user_feedback_json.RootElement.EnumerateArray()
.Select(e => int.Parse( e.GetProperty("url").GetString().Split('/').Last() ))
.Distinct()
.ToHashSet();

In [1]:
var data = from review in adventureworks.ProductReview
join product in adventureworks.Product on review.ProductId equals product.ProductId
join subCategory in adventureworks.ProductSubcategory on product.ProductSubcategoryId equals subCategory.ProductSubcategoryId
where review.Comments.Contains("<script")
select new { product.ProductId
    ,product.Name
    ,product.ProductNumber
    ,subCategory.ProductSubcategoryId
    ,subCategory.ProductCategoryId
    ,category = $"{subCategory.ProductCategoryId}-{subCategory.ProductSubcategoryId}"
    ,CommentLength = review.Comments.Length
    ,review.Comments
};

In [1]:
data.ExploreWithSandDance().Display();

# results
Hi Jon run the notebook, we have found the root cause, we are exposed to js injection and the affected pages are more than the number from the report


![ppap](https://media4.giphy.com/media/CEF7ocyS0C9q/giphy.gif)

In [1]:
data.Count()